Create silence class

In [ ]:
import os
import warnings

import librosa
import numpy as np
import soundfile as sf

warnings.filterwarnings("ignore", category=DeprecationWarning)
AUDIO_FORMAT = "WAV"

In [ ]:
train_audio_path = 'train/audio'
train_audio_path_without_silence = 'train/audio_without_silence'

class_names = []

for file in os.listdir(train_audio_path_without_silence):
    d = os.path.join(train_audio_path_without_silence, file)
    if os.path.isdir(d):
        class_names.append(str(os.path.basename(d)))

### Silence from background_noise

In [ ]:
import librosa
import soundfile as sf

class_name = f"./_background_noise_"

class_path = f"{train_audio_path}/{class_name}"
class_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

len_of_window = 16000
silence_iter = 0

for class_file in class_files:
    audio_file, sr = librosa.load(f"{class_path}/{class_file}", sr=None)
    i = 0
    len_of_audio = len(audio_file)
    while i < len_of_audio:
        part_of_audio = audio_file[i:i+len_of_window] 
        sf.write(f"./silence_background_noise/silence_{silence_iter}.wav", part_of_audio, sr, 'PCM_16')
    
        silence_iter += 1
        i += len_of_window


### Silence from all audios

In [ ]:
def complement_of_intervals(total_set, intervals):
    result = [total_set]

    for interval in intervals:
        for i, (start, end) in enumerate(result):
            if start <= interval[0] <= end:
                result.pop(i)
                if start < interval[0]:
                    result.insert(i, [start, interval[0]])
                    i += 1
                if interval[1] < end:
                    result.insert(i, [interval[1], end])
            elif start <= interval[1] <= end:
                result.pop(i)
                if interval[0] > start:
                    result.insert(i, [start, interval[0]])
                    i += 1
                if interval[1] < end:
                    result.insert(i, [interval[1], end])
    return result

In [ ]:
import librosa
import soundfile as sf

flag_stop = False
n_fft = 1024
hop_length = 256
silence_dir = f"./{train_audio_path_without_silence}/silence"

silence_iter = 0
non_silence = []
for class_name in class_names:
    if flag_stop:
        break
    class_path = f"{train_audio_path}/{class_name}"
    
    class_files = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
    print(class_name)
    for class_file in class_files:
        if flag_stop:
            break

        audio_file, sr = librosa.load(f"{class_path}/{class_file}", sr=None)

        energy = librosa.feature.rms(y=audio_file, frame_length=n_fft, hop_length=hop_length)
        silence_threshold = 0.5 * max(energy)
        nonsilent_segments = librosa.effects.split(audio_file, top_db=40, hop_length=hop_length)
        silent_segments = complement_of_intervals([0, len(audio_file)], nonsilent_segments)

        while len(non_silence) < int(16000 / 256):
            for i, (start, end) in enumerate(silent_segments):
                non_silence.append(audio_file[start:end])
            if len(non_silence) == 0:
                break
        
        if len(non_silence) != 0:
            concatenated = np.concatenate(non_silence, axis=0)

            sf.write(f"./silence/silence_{silence_iter}.wav", concatenated[0:16000], sr, 'PCM_16')
            silence_iter += 1
            non_silence = []
        if silence_iter > 2000:
            flag_stop = True

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6ec8773e-c89e-4114-ab03-f7a026dc9b15' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>